In [6]:
import numpy as np

In [2]:
"""
X = [[x_00, x_01, x_02],
     [x_10, x_11, x_12],
     [x_20, x_21, x_22]]

K = [[w_00, w_01],
     [w_10, w_11]]

grad_output = a  b
              c  d

Y = y_00 y_01
    y_10 y_11

y_00 = x_00 * w_00 + x_01 * w_01 + x_10 * w_10 + x_11 * w_11
y_01 = x_01 * w_00 + x_02 * w_01 + x_11 * w_10 + x_12 * w_11
y_10 = x_10 * w_00 + x_11 * w_01 + x_20 * w_10 + x_21 * w_11
y_11 = x_11 * w_00 + x_12 * w_01 + x_21 * w_10 + x_22 * w_11

∂L / ∂w_00 = a * x_00 + b * x_01 + c * x_10 + d * x_11
∂L / ∂w_01 = a * x_01 + b * x_02 + c * x_11 + d * x_12
∂L / ∂w_10 = a * x_10 + b * x_11 + c * x_20 + d * x_21
∂L / ∂w_11 = a * x_11 + b * x_12 + c * x_21 + d * x_22

--------------------------------------------------------

∂L / ∂x_00 = a * w_00
∂L / ∂x_01 = a * w_01 + b * w_00
∂L / ∂x_02 = b * w_01
∂L / ∂x_10 = a * w_10 + c * w_00
∂L / ∂x_11 = a * w_11 + b * w_10 + c * w_01 + d * w_00
∂L / ∂x_12 = b * w_11 + d * w_01
∂L / ∂x_20 = c * w_10
∂L / ∂x_21 = c * w_11 + d * w_10
∂L / ∂x_22 = d * w_11

                                 
fliplr =  w_01 w_00     flipud =  w_11 w_10
          w_11 w_10               w_01 w_00


grad_output_padded = 0  0  0  0
                     0  a  b  0
                     0  c  d  0
                     0  0  0  0

grad_input = a * w_00          a * w_01 + b * w_00                  b * w_01
             a*w_10 + c*w_00   a*w_11 + b*w_10 + c*w_01 + d*w_00  b*w_11 + d*w_01
             c*w_10            c*w_11 + d*w_10                       d*w_11


"""

class Conv2d:
    def __init__(self, kernel_size=3):
        self.kernel = np.random.randn(kernel_size, kernel_size) * 0.1

    def forward(self, x):
        self.input = x
        h, w = x.shape
        k_h, k_w = self.kernel.shape
        output_h = h - k_h + 1
        output_w = w - k_w + 1
        output = np.zeros((output_h, output_w))

        for i in range(output_h):
            for j in range(output_w):
                region = x[i:i+k_h, j:j+k_w]
                output[i, j] = np.sum(region * self.kernel)

        return output

    def backward(self, grad_output):
        h, w = self.input.shape
        k_h, k_w = self.kernel.shape
        o_h, o_w = grad_output.shape

        self.grad_weights = np.zeros_like(self.kernel)
        
        for i in range(o_h):
            for j in range(o_w):
                patch = self.input[i:i+k_h, j:j+k_w]
                self.grad_weights += grad_output[i, j] * patch

        flipped = np.flipud(np.fliplr(self.kernel))
        padded_grad = np.pad(grad_output, ((k_h-1, k_h-1), (k_w-1, k_w-1)), 'constant')
        
        grad_input = np.zeros((h, w))
        
        for i in range(h):
            for j in range(w):
                region = padded_grad[i:i+k_h, j:j+k_w]
                grad_input[i,j] = np.sum(region * flipped)
        return grad_input

In [3]:
class Relu:
    def forward(self, x):
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad_input = grad_output.copy()
        grad_input[self.input <= 0] = 0
        return grad_input
        

In [4]:
"""
Dense1 -> Dense2 -> MSE

For Dense1
----
o1 = w1 * x1 + b1
o2 = w2 * (w1 * x1 + b1) + b2
L = 1/2 (o2 - y) ^ 2

∂L / ∂w1 = ∂L / ∂o2 * ∂o2 / ∂o1 * ∂o1 / ∂w1
∂L / ∂o2 = o2 - y
∂o2 / ∂o1 = w2
∂o1 / ∂w1 = x1

∂L / ∂w1 = (o2 - y) * w2 * x1
            |          |
        grad_output for dense 1
∂L / ∂w2 = (o2 - y) * x2 
           |       | 
        grad_output for dense 2
"""

class Dense:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size) * 0.1
        self.biases = np.random.randn(output_size) * 0.1

    def forward(self, x):
        self.input = x
        return self.weights @ x + self.biases

    def backward(self, grad_output):
        self.grad_weights = np.outer(grad_output, self.input)
        self.grad_biases = grad_output
        
        return self.weights.T @ grad_output



In [5]:
"""
Pooling
---
X= 1  5  4
   9  2  1
   6  3  2

kernel_size = 2,2

Max Pooling
--
Y = 9 5
    9 3

|grad_output| = a b
                c d

for
1  5
9  2      we have a

for second window b, third c, fourth d
So,
∂L / ∂X = 0     b  0 
          a + c 0  0
          0     d  0
          
Avg Pooling
--
Y = 4.25 3
    5 2

for avg we have to distribute because each value contributes equally

∂L / ∂X = a/4          a/4 + b/4                b/4 
          a/4 + c/4    a/4 + b/4 + c/4 + d/4    b/4 + d/4
          c/4          c/4 + d/4                d/4

"""


class Pooling:
    def __init__(self, pool_type='max', kernel_size=(3, 3)):
        self.pool_type = pool_type
        self.kernel_size = kernel_size
        self.indicies = []

    def forward(self, x):
        self.input_shape = x.shape
        h, w = x.shape
        kh, kw = self.kernel_size
        output_h = h - kh + 1
        output_w = w - kw + 1
        output = np.zeros((output_h, output_w))

        self.indices = []

        for i in range(output_h):
            for j in range(output_w):
                region = x[i:i+kh, j:j+kw]
                if self.pool_type == 'max':
                    output[i, j] = np.max(region)
                    max_index = np.unravel_index(np.argmax(region), region.shape)
                    absolute_index = (i + max_index[0], j + max_index[1])
                    self.indices.append(absolute_index)
                elif self.pool_type == 'avg':
                    output[i, j] = np.mean(region)
        
        return output

    def backward(self, grad_output):
        grad_input = np.zeros(self.input_shape)
        kh, kw = self.kernel_size
        output_h, output_w = grad_output.shape
        
        if self.pool_type == 'max':
            for idx, (x_i, x_j) in enumerate(self.indices):
                i = idx // output_w
                j = idx % output_w
                grad_input[x_i, x_j] += grad_output[i, j]

        elif self.pool_type == 'avg':
            for i in range(output_h):
                for j in range(output_w):
                    grad_input[i:i+kh, j:j+kw] += grad_output[i, j] / (kh * kw)
        return grad_input


In [26]:
class Flatten:
    def forward(self, x):
        self.input_shape = x.shape
        return x.flatten()
        
    def backward(self, grad_output):
        return grad_output.reshape(self.input_shape)

In [32]:
class Model:
    def __init__(self, layers):
        self.layers = layers

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
        
    def backward(self, grad_output):
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output)

In [28]:
class SoftmaxCrossEntropyLoss:
    def forward(self, logits, labels):
        self.probs = self._softmax(logits)
        self.labels = labels
        loss = -np.sum(labels * np.log(self.probs + 1e-9)) / logits.shape[0]
        return loss

    def backward(self):
        return (self.probs - self.labels) / self.labels.shape[0]

    def _softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / np.sum(e_x, axis=1, keepdims=True)


In [29]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1, as_frame=False)

X = mnist['data']
y = mnist['target'].astype(np.int32)

X = X / 255.0

print(X.shape)

X = X.reshape(-1, 28, 28)
print(X.shape)

(70000, 784)
(70000, 28, 28)


# Tackling Around

In [97]:
a = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9],
])

kernel = np.array([
    [0, 1, 0],
    [1, 0, 0]
    
])

grad_output = np.array([
    [10],
    [20]
])

kh, kw = grad_output.shape
out_h, out_w = grad_output.shape

o1 = np.zeros_like(kernel)
o2 = np.zeros_like(kernel)

for i in range(kh):
    for j in range(kw):
        window = a[i:i + kh, j:j + kw]
        # o1[i,j] = np.tensordot(window, grad_output)
        o1[i,j] = np.sum(window * grad_output)

for i in range(kh):
    for j in range(kw):
        for m in range(out_h):
            for n in range(out_w):
                o2[i, j] += a[m + i, n + j] * grad_output[m, n]

o3 = np.zeros_like(kernel)

H_k, W_k = kernel.shape
H_out, W_out = grad_output.shape

for i in range(H_out):
    for j in range(W_out):
        patch = a[i:i+H_k, j:j+W_k]
        o3 += grad_output[i, j] * patch

print(o1)
print(o2)
print(o3) # correct generalization


h, w = a.shape
kh, kw = kernel.shape


flipped = np.flipud(np.fliplr(kernel))
padded_grad = np.pad(grad_output, ((kh-1, kh-1), (kw-1, kw-1)), 'constant')

grad_input = np.zeros((h, w))

for i in range(h):
    for j in range(w):
        region = padded_grad[i:i+kh, j:j+kw]
        grad_input[i,j] = np.sum(region * flipped)


"""
[[0 0 0 0]
 [0 1 2 0]
 [0 3 4 0]
 [0 0 0 0]]

 
[[0 1]
 [1 0]]
"""
grad_input


[[ 90   0   0]
 [180   0   0]]
[[ 90   0   0]
 [180   0   0]]
[[ 90 120 150]
 [180 210 240]]


array([[ 0., 10.,  0.],
       [10., 20.,  0.],
       [20.,  0.,  0.]])

In [18]:
a = np.array([
    [1,5,4,8],
    [9,2,1,12],
    [6,3,2,9]
])

def backward(grad_output, input_shape, kernel_size, indices, pool_type):
    grad_input = np.zeros(input_shape)
    kh, kw = kernel_size
    output_h, output_w = grad_output.shape
    
    if pool_type == 'max':
        for idx, (x_i, x_j) in enumerate(indices):
            i = idx // output_w
            j = idx % output_w
            grad_input[x_i, x_j] += grad_output[i, j]

    elif pool_type == 'avg':
        for i in range(output_h):
            for j in range(output_w):
                grad_input[i:i+kh, j:j+kw] += grad_output[i, j] / (kh * kw)

    return grad_input

b = np.array([
    [1, 2, 3],
    [4, 5, 6]
    
])
backward(b, (3,4), (2,2), [(1,0), (0,1), (1,3), (1,0), (2,1), (1,3)], "max")


array([[0., 2., 0., 0.],
       [5., 0., 0., 9.],
       [0., 5., 0., 0.]])

In [36]:
           
# XOR data
X = np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
]).T  # shape (2,4) for convenience

Y = np.array([0,1,1,0])  # target outputs

# Hyperparameters
lr = 0.1
epochs = 50000

# Create model: 2 inputs -> 2 hidden units -> 1 output
model = Model([
    Dense(2,2),
    Sigmoid(),
    Dense(2,1),
    Sigmoid()
])

for epoch in range(epochs):
    loss = 0
    for i in range(X.shape[1]):
        x = X[:,i]
        y = Y[i]
        
        # Forward
        out = model.forward(x)
        
        # Compute loss (MSE)
        loss += 0.5 * (out - y)**2
        
        # Backward
        grad = out - y  # dL/dout for MSE
        model.backward(grad)
        
        # Update weights
        for layer in model.layers:
            if isinstance(layer, Dense):
                layer.weights -= lr * layer.grad_weights
                layer.biases -= lr * layer.grad_biases
                
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss {loss}")

# Test model
for i in range(X.shape[1]):
    x = X[:,i]
    out = model.forward(x)
    print(f"Input: {x}, Predicted: {out[0]:.3f}, Target: {Y[i]}")


Epoch 0, Loss [0.50454905]
Epoch 500, Loss [0.50448421]
Epoch 1000, Loss [0.50443864]
Epoch 1500, Loss [0.5043954]
Epoch 2000, Loss [0.50435437]
Epoch 2500, Loss [0.50431538]
Epoch 3000, Loss [0.50427821]
Epoch 3500, Loss [0.50424256]
Epoch 4000, Loss [0.50420805]
Epoch 4500, Loss [0.50417417]
Epoch 5000, Loss [0.50414025]
Epoch 5500, Loss [0.50410536]
Epoch 6000, Loss [0.50406812]
Epoch 6500, Loss [0.5040264]
Epoch 7000, Loss [0.50397657]
Epoch 7500, Loss [0.50391207]
Epoch 8000, Loss [0.50381979]
Epoch 8500, Loss [0.5036707]
Epoch 9000, Loss [0.50339147]
Epoch 9500, Loss [0.50276497]
Epoch 10000, Loss [0.50102917]
Epoch 10500, Loss [0.49529516]
Epoch 11000, Loss [0.47696605]
Epoch 11500, Loss [0.43548686]
Epoch 12000, Loss [0.38888508]
Epoch 12500, Loss [0.35367412]
Epoch 13000, Loss [0.31129782]
Epoch 13500, Loss [0.22513766]
Epoch 14000, Loss [0.11848353]
Epoch 14500, Loss [0.06158261]
Epoch 15000, Loss [0.03733865]
Epoch 15500, Loss [0.02562151]
Epoch 16000, Loss [0.01908515]
Epoc